In [1]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

In [2]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 8
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 60000
eval_mode_after: -1
learning_rate: 0.0001
print_every: 100
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender



In [3]:
vocab, train_loader, val_loader = build_loaders(args)

Starting to read the json file for SUNCG
loading relation score matrix from:  new/relation_graph_v1.p
Training dataset has 53860 scenes and 708041 objects
(13.15 objects per image)
Starting to read the json file for SUNCG
loading relation score matrix from:  new/relation_graph_v1.p


In [4]:
dt = train_loader.dataset

In [5]:
model, model_kwargs = build_model(args, vocab)

In [6]:
print(model)
model.float().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

Sg2ScVAEModel(
  (obj_embeddings_ec): Embedding(33, 48)
  (pred_embeddings_ec): Embedding(16, 128)
  (obj_embeddings_dc): Embedding(33, 48)
  (pred_embeddings_dc): Embedding(16, 128)
  (attr_embedding_ec): Embedding(5, 16)
  (attr_embedding_dc): Embedding(5, 16)
  (box_embeddings): Linear(in_features=6, out_features=48, bias=True)
  (angle_embeddings): Embedding(24, 16)
  (box_mean_var): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (box_mean): Sequential(
    (0): Linear(in_features=128, out_features=48, bias=True)
  )
  (box_var): Sequential(
    (0): Linear(in_features=128, out_features=48, bias=True)
  )
  (angle_mean_var): Sequential(
    (0): Linear(in_features=128, out_feature

In [7]:
for batch in train_loader:
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    break

In [10]:
model_out = model(objs, triples, boxes, angles, attributes, obj_to_img)
mu, logvar, boxes_pred, angles_pred = model_out

In [12]:
model.decoder_cat

True

In [13]:
mu.shape

torch.Size([240, 64])

In [14]:
objs.shape

torch.Size([240])

In [18]:
model.pred_embeddings_ec

Embedding(16, 128)

In [19]:
torch.__version__

'1.8.1+cu111'

In [20]:
torch.version.cuda

'11.1'